In [2]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv("./data/titanic.csv")
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 1. Build Model using titanic dataset 

Use the Pclass, Sex, Age, SibSp, Fare as X <br />
<br />
Survived column as y
<br 

Solve above classification problem also use standard scalar to process the input X data<br />
For training the model use 80:20 split for training and test data respetively.<br />
Calculate the accuracy of the model using sklearn - metrics packages<br />

In [3]:
dataset.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
titanic_dataset = dataset[["Pclass", "Sex", "Age", "SibSp", "Fare", "Survived"]]
titanic_dataset.head()

,Pclass,Sex,Age,SibSp,Fare,Survived
0,3,male,22.0,1,7.2500,0
1,1,female,38.0,1,71.2833,1
2,3,female,26.0,0,7.9250,1
3,1,female,35.0,1,53.1000,1
4,3,male,35.0,0,8.0500,0


In [5]:
# Make a copy for slicing
titanic_dataset = titanic_dataset.copy()

In [6]:
def change_data(x):
    if x == "male":
        return 1
    else:
        return 0

In [7]:
titanic_dataset["Sex"] = titanic_dataset["Sex"].apply(change_data)

In [8]:
titanic_dataset["Fare"] = titanic_dataset["Fare"].apply(lambda x: "%.2f" % x)

In [9]:
titanic_dataset.head()

,Pclass,Sex,Age,SibSp,Fare,Survived
0,3,1,22.0,1,7.25,0
1,1,0,38.0,1,71.28,1
2,3,0,26.0,0,7.92,1
3,1,0,35.0,1,53.10,1
4,3,1,35.0,0,8.05,0


In [10]:
# Fill the Median age in NaN rows
median_age = titanic_dataset["Age"].median()

In [11]:
# Fill the age column with the median age
titanic_dataset.loc[((titanic_dataset["Age"].isna()) | (titanic_dataset["Age"] == 0)), "Age"] = median_age

In [17]:
# Change the datatype of Age and Fare
titanic_dataset["Age"] = titanic_dataset["Age"].astype(int)
titanic_dataset["Fare"] = titanic_dataset["Fare"].astype(float)

In [18]:
## Get X - IDV's and y- DV's
X = titanic_dataset.iloc[:, :-1].values
y = titanic_dataset.iloc[:, -1].values

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

In [22]:
# Standard scaler to scale the data to -1 to 1 range
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [24]:
X_train[:5]

array([[ 0.84105013, -1.39360997, -0.17205357, -0.44991607, -0.41613941],
       [-0.36626377,  0.71756088,  1.7198714 , -0.44991607, -0.38840204],
       [-0.36626377,  0.71756088, -0.17205357, -0.44991607, -0.3790902 ],
       [ 0.84105013,  0.71756088,  0.06443705, -0.44991607, -0.49301158],
       [ 0.84105013,  0.71756088, -0.09322336,  6.87214654,  0.74130173]])

In [25]:
# K-Nearest Neighbour
from sklearn.neighbors import KNeighborsClassifier
# minkowski is for ecledian distance
classifier = KNeighborsClassifier(n_neighbors = 5, metric='minkowski', p=2)

In [26]:
# Model training
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [27]:
# Predict on the test data
y_pred = classifier.predict(X_test)
y_prob = classifier.predict_proba(X_test)[:,1]

In [28]:
# Confusion metrics
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[84, 16],
       [30, 49]])

In [29]:
# Calculate the accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.7430167597765364


In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.84      0.79       100
           1       0.75      0.62      0.68        79

    accuracy                           0.74       179
   macro avg       0.75      0.73      0.73       179
weighted avg       0.74      0.74      0.74       179



In [39]:
## Test on New data
new_pred = classifier.predict(sc.transform(np.array([[2, 1, 24, 1, 8.12]])))[0]
new_pred

0

## Save the model & standard scaler to a pickle file

In [41]:
import pickle

In [42]:
# Save the standard scaler
scaler_file = "sc.pickle"
pickle.dump(sc, open(scaler_file,'wb'))

In [43]:
# save the classifier object
model_file = "classifier.pickle"
pickle.dump(classifier, open(model_file,'wb'))

## 2. Deploy the model

Use the standard scaler pickle file and classfier pickle file. <br />
Create an API using Flask.<br />

In [45]:
# Refer the rest_api_server.py code
# Refer the rest_client,py

## Restaurant Reviews _ NLP Problem

1. Build the Sentiment classification model using the Restaurant reviews dataset
2. Use Tf-idf vectorizer function to process the data
3. Save the Tf-idf vectorized and classifier (KNN) pickle file
4. Deploy the model as an API using Flask in local machine

In [3]:
reviews_dataset = pd.read_csv("./data/Restaurant_Reviews.tsv.txt", delimiter="\t")

In [4]:
reviews_dataset.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
reviews_dataset.shape

(1000, 2)

In [10]:
# Downlad all the neccessary packages
import nltk
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/arunachalamdharmaraj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [12]:
# Stemming - to the root word
ps = PorterStemmer()

In [14]:
corpus = []
for i in range(len(reviews_dataset)):
    customer_review = re.sub('[^a-zA-Z]', ' ',reviews_dataset['Review'][i])
    customer_review = customer_review.lower()
    customer_review = customer_review.split()
    clean_review = [ps.stem(word) for word in customer_review if not word in set(stopwords.words('english'))]
    clean_review = ' '.join(clean_review)
    corpus.append(clean_review)

In [16]:
corpus[0], corpus[1]

('wow love place', 'crust good')

## 2. Build the Tf-idf vectorizer

In [17]:
# TF-idf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 1500, min_df = 3, max_df = 0.6)

In [18]:
X = vectorizer.fit_transform(corpus).toarray()

In [20]:
y = reviews_dataset.iloc[:, 1].values

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [25]:
from sklearn.neighbors import KNeighborsClassifier
classifierKNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifierKNN.fit(X_train, y_train)

KNeighborsClassifier()

In [26]:
# Prediction
y_pred_knn = classifierKNN.predict(X_test)

In [27]:
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix

cmknn = confusion_matrix(y_test, y_pred_knn)
cmknn

array([[92,  5],
       [84, 19]])

In [28]:
print("KNN accuracy \n", accuracy_score(y_test,y_pred_knn))

KNN accuracy 
 0.555


## Test on new data

In [30]:
sample = ["Good batting by England"]
sample = vectorizer.transform(sample).toarray()

sentiment = classifierKNN.predict(sample)
sentiment

array([1])

## 3. Save the Pickle files

In [31]:
import pickle

with open('textclassifier.pickle','wb') as file:
    pickle.dump(classifierKNN,file)

with open('tfidfmodel.pickle','wb') as file:
    pickle.dump(vectorizer,file)


## 4. Deploy the model using Flask 

In [33]:
## Refer the nlp_api_client.py
## Refer the nlp_api_server.py